# Identify gaps in CARE service relative to unsheltered population
* Map number of CARE/CARE+ requests by tract (2017-2019)
* Map number of homeless counts by tract (2017-2019)
* Map difference: (total CARE/CARE+ for homeless encampments) - (total homelessness)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl, basemaps
import json
import requests
from ipywidgets import link, FloatSlider, Text, HTML
from branca.colormap import linear
import boto3
import folium

In [2]:
catalog = intake.open_catalog('../catalogs/*.yml')

bucket_name = 's3://city-of-los-angeles-data-lake/public-health-dashboard/'

s3 = boto3.client('s3')

In [10]:
import s3fs
import boto3
print(s3fs.__version__)
print(boto3.__version__)

0.3.5
1.9.249


In [7]:
pd.read_parquet('s3://city-of-los-angeles-data-lake/public-health-dashboard/gis/intermediate/homelessness_lacity_2017_2019.parquet')

PermissionError: Access Denied

## Import files

In [3]:
homeless = pd.read_parquet(f'{bucket_name}gis/intermediate/homelessness_lacity_2017_2019.parquet')

PermissionError: Access Denied

In [4]:
care = gpd.read_file(f'{bucket_name}gis/intermediate/care311_tracts.geojson')

In [ ]:
df = pd.merge(homeless, care, on = ['GEOID', 'year', 'geometry', 'full_area', 'clipped_area'], how = 'left', validate = '1:1')

In [4]:
homeless = gpd.read_file(f'{bucket_name}gis/raw/homelessness_lacity_2017_2019.geojson')

In [3]:
care = gpd.read_file(f'{bucket_name}gis/intermediate/care311_tracts.geojson').to_crs({'init':'epsg:4326'})

## Merge dfs

In [5]:
# Merge in CARE 311 service requests
m1 = pd.merge(homeless, care, on = ['GEOID', 'year'], how = 'left', validate = '1:1')

KeyError: 'year'

In [ ]:


# Keep the geometry from homelessness because it's been clipped to City of LA
m2 = m2.drop(columns = ['geometry_y'])

m2.rename(columns = {'geometry_x': 'geometry', 'GEOID': 'id'}, inplace = True)

# Fill in NaNs with zeroes for CARE service requests
for col in ['bulky', 'homeless', 'illegal', 'other']:
    m2[col] = m2[col].fillna(0)    

pivot1 = m2.dissolve(by = ['id', 'clipped_area', 'full_area', 
                           'CD', 'pop', 'SPA', 'SD'], aggfunc = 'sum').reset_index().drop(columns = ['year'])

pivot1['UnsheltDensity'] = pivot1.totUnshelt / pivot1.clipped_area
pivot1['CARE_Unshelt'] = pivot1.apply(lambda row: row.homeless / row.totUnshelt if row.totUnshelt > 0 else np.nan, axis = 1)
pivot1['Unshelt_CARE'] = pivot1.apply(lambda row: row.totUnshelt / row.homeless if row.homeless > 0 else np.nan, axis = 1)

pivot1.to_file(driver = 'GeoJSON', filename = '../gis/homelessness_tract.geojson')
s3.upload_file('../gis/homelessness_tract.geojson', 'city-of-los-angeles-data-lake', 
               'public-health-dashboard/gis/intermediate/homelessness_tract.geojson')
"""

In [3]:
pivot1 = gpd.read_file(f'{bucket_name}gis/intermediate/homelessness_tract.geojson')

In [4]:
pivot1.head()

,id,clipped_area,full_area,CD,pop,SPA,SD,totUnshelt,totShelt,totPeople,bulky,homeless,illegal,other,UnsheltDensity,CARE_Unshelt,Unshelt_CARE,geometry
0,06037101110,0.441019,0.441019,7,4566.0,2,5,29.498,0,29.498,1.0,3.0,0.0,0.0,66.885944,0.101702,9.832667,"POLYGON ((-118.30229 34.25870, -118.30091 34.2..."
1,06037101122,1.020722,1.020722,7,3064.0,2,5,4.618,0,4.618,0.0,0.0,0.0,0.0,4.524246,0.000000,NaN,"POLYGON ((-118.30334 34.27371, -118.30330 34.2..."
2,06037101210,0.251197,0.251197,7,6043.0,2,5,41.897,0,41.897,1.0,3.0,6.0,1.0,166.789457,0.071604,13.965667,"POLYGON ((-118.29945 34.25598, -118.29792 34.2..."
3,06037101220,0.269802,0.269802,7,3340.0,2,5,72.464,0,72.464,2.0,1.0,1.0,0.0,268.581682,0.013800,72.464000,"POLYGON ((-118.28593 34.25227, -118.28592 34.2..."
4,06037101300,0.995075,0.996474,7,4285.0,2,5,36.710,0,36.710,2.0,8.0,5.0,0.0,36.891706,0.217924,4.588750,"POLYGON ((-118.27822 34.25068, -118.27822 34.2..."


## Map homelessness

In [5]:
tracts = pivot1.copy()
tracts['GEOID'] = tracts['id']
tracts = tracts.set_index('id')
tracts.head()

,clipped_area,full_area,CD,pop,SPA,SD,totUnshelt,totShelt,totPeople,bulky,homeless,illegal,other,UnsheltDensity,CARE_Unshelt,Unshelt_CARE,geometry,GEOID
id,,,,,,,,,,,,,,,,,,
06037101110,0.441019,0.441019,7,4566.0,2,5,29.498,0,29.498,1.0,3.0,0.0,0.0,66.885944,0.101702,9.832667,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",06037101110
06037101122,1.020722,1.020722,7,3064.0,2,5,4.618,0,4.618,0.0,0.0,0.0,0.0,4.524246,0.000000,NaN,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",06037101122
06037101210,0.251197,0.251197,7,6043.0,2,5,41.897,0,41.897,1.0,3.0,6.0,1.0,166.789457,0.071604,13.965667,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",06037101210
06037101220,0.269802,0.269802,7,3340.0,2,5,72.464,0,72.464,2.0,1.0,1.0,0.0,268.581682,0.013800,72.464000,"POLYGON ((-118.28593 34.25227, -118.28592 34.2...",06037101220
06037101300,0.995075,0.996474,7,4285.0,2,5,36.710,0,36.710,2.0,8.0,5.0,0.0,36.891706,0.217924,4.588750,"POLYGON ((-118.27822 34.25068, -118.27822 34.2...",06037101300


In [6]:
# to_json converts the gdf to json. json.loads converts it into dictionary 
# geo_data can have many more columns than just geometry. Need to include other columns if you want to have the HTML popup include info.
geo_data = json.loads(tracts.to_json())

# Take what we want to map and turn it into a dictionary
# Can only include the key-value pair, the value you want to map, nothing more.
unsheltered = dict(zip(pivot1['id'].tolist(), pivot1['totUnshelt'].tolist()))
unshelt_density = dict(zip(pivot1['id'].tolist(), pivot1['UnsheltDensity'].tolist()))
unshelt_CARE = dict(zip(pivot1['id'].tolist(), pivot1['Unshelt_CARE'].tolist()))
CARE_unshelt = dict(zip(pivot1['id'].tolist(), pivot1['CARE_Unshelt'].tolist()))

In [7]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10,
                  basemap = basemaps.CartoDB.Positron)

layer = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = unsheltered,
    colormap = linear.viridis,
    border_color = 'white',
    style = {'fillOpacity': 0.6, 'weight': 0.8, 'opacity': 0.6},
    hover_style = {'fillOpacity': 0.8},
    value_min = 0,
    value_max = 100
)

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def update_html(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        Total Unsheltered:
        {} 
    '''.format(id, feature['properties']['totUnshelt'])
 
    
layer.on_hover(update_html)

control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
m.add_layer(layer)
m.add_control(control)

m

Map(basemap={'url': 'http://c.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png', 'max_zoom': 20, 'attribution':…

In [8]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10,
                  basemap = basemaps.CartoDB.Positron)

layer = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = unshelt_density,
    colormap = linear.Oranges_08,
    border_color = 'white',
    style = {'fillOpacity': 0.7, 'weight': 0.8, 'opacity': 0.6},
    hover_style = {'fillOpacity': 0.8},
    value_min = 0,
    value_max = 500
)

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def update_html(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        Unsheltered per Square Mile:
        {} 
    '''.format(id, feature['properties']['UnsheltDensity'])
 
    
layer.on_hover(update_html)

control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
m.add_layer(layer)
m.add_control(control)

m

Map(basemap={'url': 'http://c.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png', 'max_zoom': 20, 'attribution':…

In [ ]:
unshelt_df = pivot1[['id', 'totUnshelt']]
unshelt_df.rename(columns = {'id': 'GEOID'}, inplace = True)
#geo = pivot1[['id', 'geometry']]
 
m = folium.Map(
    location=[34.0536, -118.2427], 
    zoom_start=12, 
    tiles='cartodbpositron'
)

folium.Choropleth(
    geo_data=pivot1,
    name = 'choropleth',
    data=unshelt_df,
    columns=['GEOID', 'totUnshelt'],
    key_on='feature.properties.id',
    fill_color='BuPu',
    fill_opacity=0.95,
    line_opacity=0.8, line_color='white',
    bins=[0, 50, 100, 500, 4000],
    legend_name='Housing Units per Acre'
).add_to(m)


#m.add_child(folium.LayerControl())

m.save('map.html')